In [1]:
%matplotlib inline

# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Pillow
from PIL import Image
#tqdm
from tqdm.notebook import tqdm
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import transforms
#OS
import os

Data Loading using the dataloader class

In [2]:
class Lung_Dataset(Dataset):
    """
    Lung Dataset Consisting of Infected and Non-Infected.
    """

    def __init__(self, purpose, verbose=0):
        """
        Constructor for generic Dataset class - simply assembles
        the important parameters in attributes.
        
        Parameter:
        -purpose variable should be set to a string of either 'train', 'test' or 'val'
        -verbose takes an int of either 0,1 or 2. 0 will only differentiate between normal and infected, 1 will differentiate
            between normal, covid and non-covid while 2 will only differentiate between covid and non-covid
        """
        self.purpose = purpose
        self.verbose = verbose
        
        # All images are of size 150 x 150
        self.img_size = (150, 150)
            
        # The dataset has been split in training, testing and validation datasets
        self.groups = ['train', 'test', 'val']
        
        # Path to images for different parts of the dataset
        self.dataset_paths = {'train_normal': './dataset/train/normal/',
                              'train_infected': './dataset/train/infected/',
                              'train_infected_covid': './dataset/train/infected/covid',
                              'train_infected_non_covid': './dataset/train/infected/non-covid',
                              'test_normal': './dataset/test/normal/',
                              'test_infected': './dataset/test/infected/',
                              'test_infected_covid': './dataset/test/infected/covid',
                              'test_infected_non_covid': './dataset/test/infected/non-covid',
                              'val_normal': './dataset/val/normal/',
                              'val_infected': './dataset/val/infected/',
                              'val_infected_covid': './dataset/val/infected/covid',
                              'val_infected_non_covid': './dataset/val/infected/non-covid'}
        
        self.dataset_numbers = {}
        
        # Consider normal and infected only
        if verbose == 0:
            self.classes = {0: 'normal', 1: 'infected'}
            
            #Populate self.dataset_numbers
            for condition in self.classes.values():
                key = "{}_{}".format(self.purpose, condition)
                if condition == "normal":
                    file_path = self.dataset_paths[key]
                    count = len(os.listdir(file_path))
                    self.dataset_numbers[key] = count
                else:
                    key1 = key + "_covid"
                    key2 = key + "_non_covid"
                    file_path1 = self.dataset_paths[key1]
                    file_path2 = self.dataset_paths[key2]
                    count1 = len(os.listdir(file_path1))
                    count2 = len(os.listdir(file_path2))
                    count = count1 + count2
                    self.dataset_numbers[key] = count
                       
        #Consider normal, covid and non-covid
        elif verbose == 1:
            self.classes = {0: 'normal', 1: 'covid', 2: 'non_covid'}
        
            #Populate self.dataset_numbers
            for condition in self.classes.values():
                if condition == "normal":
                    key = "{}_{}".format(self.purpose, condition)
                    file_path = self.dataset_paths[key]
                    count = len(os.listdir(file_path))
                    self.dataset_numbers[key] = count
                else:
                    key = "{}_infected".format(self.purpose)
                    key1 = key + "_covid"
                    key2 = key + "_non_covid"
                    file_path1 = self.dataset_paths[key1]
                    file_path2 = self.dataset_paths[key2]
                    count1 = len(os.listdir(file_path1))
                    count2 = len(os.listdir(file_path2))
                    self.dataset_numbers[key1] = count1
                    self.dataset_numbers[key2] = count2
                
        #Consider covid and non-covid
        elif verbose == 2:
            self.classes = {0: 'covid', 1 :'non_covid' }

            #Populate self.dataset_numbers
            for condition in self.classes.values():
                key = "{}_infected".format(self.purpose)
                key1 = key + "_covid"
                key2 = key + "_non_covid"
                file_path1 = self.dataset_paths[key1]
                file_path2 = self.dataset_paths[key2]
                count1 = len(os.listdir(file_path1))
                count2 = len(os.listdir(file_path2))
                self.dataset_numbers[key1] = count1
                self.dataset_numbers[key2] = count2
            
        else:
            err_msg  = "Verbose argument only takes in an int of either 0,1 or 2"
            raise TypeError(err_msg)
        
        
    def describe(self):
        """
        Descriptor function.
        Will print details about the dataset when called.
        """
        
        # Generate description
        msg = "This is the Lung {} Dataset in the 50.039 Deep Learning class project".format(self.purpose)
        msg += " in Feb-March 2021. \n"
        msg += "It contains a total of {} images, ".format(sum(self.dataset_numbers.values()))
        msg += "of size {} by {}.\n".format(self.img_size[0], self.img_size[1])
        msg += "The images are stored in the following locations "
        msg += "and each one contains the following number of images:\n"
        for key, val in self.dataset_numbers.items():
            if key != 'infected':
                file_path = self.dataset_paths[key]
            else:
                file_path = self.dataset_paths
            msg += " - {}, in folder {}: {} images.\n".format(key, file_path, val)
        print(msg)
        
        
    def open_img(self, class_val, index_val):
        """
        Opens image with specified parameters.
        
        Parameters:
        - class_val variable should be set to 'normal' or 'infected'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        
        Returns loaded image as a normalized Numpy array.
        """
        group_val = self.purpose
        err_msg = "Error - class_val variable should be set to 'normal', 'infected', 'covid' or 'non_covid'."
        assert class_val in self.classes.values(), err_msg
        
        if class_val == 'covid' or class_val == 'non_covid':
            class_val = 'infected_' + class_val
            
        max_val = self.dataset_numbers['{}_{}'.format(group_val, class_val)]
        err_msg = "Error - index_val variable should be an integer between 0 and the maximal number of images."
        err_msg += "\n(In {}/{}, you have {} images.)".format(group_val, class_val, max_val)
        assert isinstance(index_val, int), err_msg
        assert index_val >= 0 and index_val <= max_val, err_msg
        
        # Open file as before
        if class_val != "infected":
            path_to_file = '{}/{}.jpg'.format(self.dataset_paths['{}_{}'.format(group_val, class_val)], index_val)
        else:
            covid_count = len(os.listdir(self.dataset_paths['{}_{}_covid'.format(group_val, class_val)]))
            if index_val < covid_count:
                path_to_file = '{}/{}.jpg'.format(self.dataset_paths['{}_{}_covid'.format(group_val, class_val)], index_val)
            else:
                index_val = index_val - covid_count
                path_to_file = '{}/{}.jpg'.format(self.dataset_paths['{}_{}_non_covid'.format(group_val, class_val)], index_val)
        with open(path_to_file, 'rb') as f:
            im = np.asarray(Image.open(f))/255
        f.close()
        return im
    
    
    def show_img(self, class_val, index_val):
        """
        Opens, then displays image with specified parameters.
        
        Parameters:
        - class_val variable should be set to 'normal' or 'infected'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        """
        # Open image
        im = self.open_img(class_val, index_val)
        
        # Display
        plt.imshow(im)
        
    def __len__(self):
        """
        Length special method, returns the number of images in dataset.
        """
        
        # Length function
        return sum(self.dataset_numbers.values())
    
    
    def __getitem__(self, index):
        """
        Getitem special method.
        
        Expects an integer value index, between 0 and len(self) - 1.
        
        Returns the image and its label as a one hot vector, both
        in torch tensor format in dataset.
        """
        #If we only have 2 classes
        if self.verbose == 0 or self.verbose == 2:
            first_val = int(list(self.dataset_numbers.values())[0])
            if index < first_val:
                class_val = self.classes[0]
                label = torch.Tensor([1, 0])
            else:
                class_val = self.classes[1]
                index = index - first_val
                label = torch.Tensor([0, 1])
            im = self.open_img(class_val, index)
            im = transforms.functional.to_tensor(np.array(im)).float()
          
        #If we have 3 classes to consider
        elif self.verbose == 1:
            first_val = int(list(self.dataset_numbers.values())[0])
            second_val = int(list(self.dataset_numbers.values())[1])
            if index < first_val:
                class_val = self.classes[0]
                label = torch.Tensor([1, 0, 0])
            elif index >= first_val and index < first_val + second_val:
                index = index - first_val
                class_val = self.classes[1]
                label = torch.Tensor([0,1,0])
            else:
                index = index-(first_val + second_val)
                class_val = self.classes[2]
                label = torch.Tensor([0,0,1])
            im = self.open_img(class_val, index)
            im = transforms.functional.to_tensor(np.array(im)).float()
                
        else:
            raise TypeError("Verbose value is not 0,1 or 2")
        return im, label

In [3]:
"""
Data Split for First Layer classification task (normal vs infected) 
"""

fl_labels = {
0 : "Normal",
1 : "Infected"
}

fl_train = Lung_Dataset('train', verbose = 0)
fl_test = Lung_Dataset('test', verbose = 0)
fl_val = Lung_Dataset('val', verbose = 0)

"""
Data Split for Second Layer classification task (COVID vs Non-COVID) 
"""

sl_labels = {
0 : "COVID",
1 : "Non-COVID"
}

sl_train = Lung_Dataset('train', verbose = 2)
sl_test = Lung_Dataset('test', verbose = 2)
sl_val = Lung_Dataset('val', verbose = 2)

Model Construction

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        """
        Insert each layer blocks. Same architecture will be used for the first layer and second layer CNNs
        """
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Dropout(p=0.05))
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout(p=0.05))
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Dropout(p=0.05))
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Dropout(p=0.05))

        self.final = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten())

        self.fc = nn.Linear(512, 1)
        
        """
        Initialization of the Conv layers and FC layer using Kaiming initialization
        """
        self.conv1.apply(init_weights)
        self.conv2.apply(init_weights)
        self.conv3.apply(init_weights)
        self.conv4.apply(init_weights)
        self.fc.apply(init_weights)
    
    def forward(self,x):
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.final(x)
        x = self.fc(x)

        return torch.sigmoid(x)

In [5]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    
    running_loss = 0
    
    tk0 = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    
    for batch_idx, (data, target) in enumerate(tk0):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        
        output = model.forward(data)
        
        target = target.argmax(dim=1, keepdim=True).float()
        
        loss_criterion = nn.BCELoss()
        loss = loss_criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * train_loader.batch_size)))
        
        if (batch_idx + 1) % 100 == 0:
            print('Epoch:', epoch, ',Training Loss:', running_loss / 100)
            running_loss = 0
        
def validate(model, device, val_loader, loss_criterion, plot=False):
    model.eval()

    correct = 0
    val_loss = 0

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)

            target = target.argmax(dim=1, keepdim=True).float()
            
            output = model.forward(data)
            val_loss += loss_criterion(output, target).item()
            
            pred = torch.round(output)
            equal_data = torch.sum(target.data == pred).item()
            correct += equal_data

    print("Validation loss: {}".format(val_loss / len(val_loader)))
    print('Validation set accuracy: ', 100. * correct / len(val_loader.dataset), '%')
    
    return (val_loss / len(val_loader))
    
def test(model, device, test_loader, plot=False):
    model.eval()
    
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)

            target = target.argmax(dim=1, keepdim=True).float()
            
            output = model.forward(data)
            pred = torch.round(output)

            equal_data = torch.sum(target.data == pred).item()
            correct += equal_data

    print('Test set accuracy: ', 100. * correct / len(test_loader.dataset), '%')

In [ ]:
def main():
    N_EPOCH = 200
    L_RATE = 0.001
    BETAS = (0.9,0.999)
    bs_val = 32
    stop_threshold = 0.0005
    temp_loss = None
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    fl_train_loader = DataLoader(fl_train, batch_size=bs_val, shuffle=True)
    fl_val_loader = DataLoader(fl_val, batch_size=bs_val, shuffle=True)
    fl_test_loader = DataLoader(fl_test, batch_size=bs_val, shuffle=True)
    
    fl_model = CNN().to(device)
    fl_optimizer = optim.Adam(fl_model.parameters(), lr=L_RATE, betas=BETAS)
    
    print("Training the first model to classify normal and infected images")
    for epoch in range(1,N_EPOCH+1):
        train(fl_model, device, fl_train_loader, fl_optimizer, epoch)
        val_loss = validate(fl_model, device, fl_val_loader, nn.BCELoss())
        
        """
        Early Stopping mechanism (form of neural network regularization)
        """
        if temp_loss == None:
            temp_loss = val_loss
        else:
            loss_change = temp_loss - val_loss
            temp_loss = val_loss
            if abs(loss_change) < stop_threshold:
                break

    print("\n\n")
    print("Test Accuracy of the first model:")
    test(fl_model, device, fl_test_loader)

    fl_model.to("cpu")
    
    
    
    #Not very sure on this one - whether I should connect the same optimizer and optimize tgt for the both models...
    temp_loss = None
    sl_train_loader = DataLoader(sl_train, batch_size=bs_val, shuffle=True)
    sl_val_loader = DataLoader(sl_val, batch_size=bs_val, shuffle=True)
    sl_test_loader = DataLoader(sl_test, batch_size=bs_val, shuffle=True)
    
    sl_model = CNN().to(device)
    
    sl_optimizer = optim.Adam(sl_model.parameters(), lr=L_RATE, betas=BETAS)
    
    print("\n\n")
    print("Training the second model to classify COVID and non-COVID images")
    for epoch in range(1,N_EPOCH+1):
        train(sl_model, device, sl_train_loader, sl_optimizer, epoch)
        val_loss = validate(sl_model, device, sl_val_loader, nn.BCELoss())
        
        """
        Early Stopping mechanism (form of neural network regularization)
        """
        if temp_loss == None:
            temp_loss = val_loss
        else:
            loss_change = temp_loss - val_loss
            temp_loss = val_loss
            if abs(loss_change) < stop_threshold:
                break
    
    print("\n\n")
    print("Test Accuracy of the second model:")
    test(sl_model, device, sl_test_loader)
    
if __name__ == '__main__':
    main()

Training the first model to classify normal and infected images


  0%|          | 0/163 [00:00<?, ?it/s]

Epoch: 1 ,Training Loss: 0.29040430396795275
Validation loss: 0.5365023016929626
Validation set accuracy:  80.0 %


  0%|          | 0/163 [00:00<?, ?it/s]

Epoch: 2 ,Training Loss: 0.12684269366785883
Validation loss: 0.4281007647514343
Validation set accuracy:  80.0 %


...

Epoch: 9 ,Training Loss: 0.04447707700077444
Validation loss: 0.1724882870912552
Validation set accuracy:  92.0 %


  0%|          | 0/163 [00:00<?, ?it/s]

Epoch: 10 ,Training Loss: 0.033022225288441405
Validation loss: 0.4102571904659271
Validation set accuracy:  84.0 %


  0%|          | 0/163 [00:00<?, ?it/s]

Epoch: 11 ,Training Loss: 0.022875219838460907


In [ ]:
'''
Placeholder - Code to generate images and the corresponding labels
if plot == True:
        example_data = np.zeros([24, 150, 150])
        example_pred = np.zeros(24)
        
        for i in range(24):
            example_data[i] = data[i][0].to("cpu").numpy()
            example_pred[i] = pred[i].to("cpu").numpy()
                    
        for i in range(24):
            plt.subplot(5,5,i+1)
            plt.imshow(example_data[i], cmap='gray', interpolation='none')
            plt.title(fl_labels[example_pred[i]])
            plt.xticks([])
            plt.yticks([])
        plt.show()
'''